In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import sys
arguments=sys.argv

In [3]:
import torch
from torch.autograd import grad
import torch.nn as nn
from numpy import genfromtxt
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
import os
import re
import math
torch.manual_seed(1234)

In [4]:
path10="./data/DEU_Omicron_check.csv"   
path11= "./data/FRA_Omicron_check.csv"                     
path16="./data/ITA_Omicron_check.csv"                      
path12="./results/Actual_Compartments_each_Sliding_Window"
path13="./results/Checkpoints"
path14="./results/output_DINN_validation/"                                     
if not os.path.exists(path14):
    os.makedirs(path14)
if not os.path.exists(path12):
    os.makedirs(path12)
if not os.path.exists(path13):
    os.makedirs(path13)


In [5]:
def TW_SIDR(sidr_data, s, e, pop):
    d1 = pd.to_datetime(sidr_data['date'])[s:e]
    d2 = sidr_data['S'][s:e]
    d3 = sidr_data['I'][s:e]
    d4 = sidr_data['H'][s:e]
    d5 = sidr_data['D'][s:e]
    d6 = sidr_data['R'][s:e]
    d7 = sidr_data['V'][s:e]
    date = []
    S = []
    I = []
    H = []
    D = []
    R = []
    V = []
    timesteps = []
    for item in range(s, e):
        date.append(d1[item])
        S.append(d2[item])  
        I.append(d3[item]) 
        H.append(d4[item]) 
        D.append(d5[item])
        R.append(d6[item])
        V.append(d7[item])
        timesteps.append(item-s)
    sidr_data1 = np.asarray([date, timesteps, S, I, H, D, R, V]).T
    columns = ['date', 't', 'S', 'I', 'H', 'D', 'R', 'V']
    sidr_data = pd.DataFrame(sidr_data1, columns=columns)
    return sidr_data



def prepare_data(file_path, pop, window_size=90):
    sidr_data = pd.read_csv(file_path, sep=',', encoding='utf-8')
    ndays = window_size
    j = 0
    g = [None] * (len(sidr_data) - ndays)
    for i in range(0, len(sidr_data) - ndays):
        g[j] = TW_SIDR(sidr_data, i, i + ndays, pop)
        j += 1
    for i in range(len(g)):
        g[i] = g[i].reset_index(drop=True)
    return g


def csvmaker(g, file_prefix):
    #covid_cases_list = [] # create a list to store all covid_cases dataframes
    #for i in range(len(g)):
    for i in range(n, m):
        data1 = np.asarray([g[i].S, g[i].I, g[i].H, g[i].D, g[i].R, g[i].V]).T
        columns = ['Susceptible', 'Infected', 'Hospitalized', 'Deaths', 'Recovered', 'Vaccinated']
        data1 = pd.DataFrame(data1, columns=columns)
        data1.to_csv(f"{path12}/{file_prefix}_{i}.csv")#, index=False)
        covid_cases = pd.read_csv(f"{path12}/{file_prefix}_{i}.csv", delimiter=',')
        covid_cases.columns = ['t', 'susceptible','infected','hospitalized', 'dead', 'recovered','vaccinated'] #rename columns
        covid_cases['t'] = covid_cases['t'].astype(float)
    return covid_cases






In [6]:
def datatotimesteps(covid_cases):

    susceptible = []
    infected = []
    hospitalized = []
    
    dead = []
    recovered = []
    vaccinated = []
    timesteps = []

    length = len(covid_cases['t'])
    d1 = covid_cases['susceptible'][:length]
    d2 = covid_cases['infected'][:length]
    d3 = covid_cases['hospitalized'][:length]
    d4 = covid_cases['dead'][:length]
    d5 = covid_cases['recovered'][:length]
    d6 = covid_cases['vaccinated'][:length]
    d7 = covid_cases['t'][:length]

    for item in range(len(d7)):
        susceptible.append(d1[item])
        infected.append(d2[item])
        hospitalized.append(d3[item])
        dead.append(d4[item])
        recovered.append(d5[item])
        vaccinated.append(d6[item])
        timesteps.append(d7[item])
    return timesteps, susceptible, infected, hospitalized, dead, recovered, vaccinated


In [7]:
class DINN(nn.Module):
    def __init__(self, t, S_data, I_data, H_data, D_data, R_data, V_data, path1, lambda_val):
        super(DINN, self).__init__()
        for country_name, country_data in [("France", data_FRA), ("Italy", data_ITA), ("Germany", data_DEU)]:
            if country_name == "Germany":
                pop = pop_DEU
            elif country_name == "France":
                pop = pop_FRA
            elif country_name == "Italy":
                pop = pop_ITA
        
        self.N = pop  # population size
        
        # Initialize tensors
        self.t = torch.tensor(t, requires_grad=True).float()
        self.t_batch = torch.reshape(self.t, (len(self.t), 1))  # reshape for batch
        self.S = torch.tensor(S_data)
        self.I = torch.tensor(I_data)
        self.H = torch.tensor(H_data)
        self.D = torch.tensor(D_data)
        self.R = torch.tensor(R_data)
        self.V = torch.tensor(V_data)


        # Save path, losses, and lambda
        self.losses = []
        self.save = 3  # file to save
        self.lambda_mse = lambda_val

        # Normalization values
        self.S_max = max(self.S)
        self.I_max = max(self.I)
        self.H_max = max(self.H)
        self.D_max = max(self.D)
        self.R_max = max(self.R)
        self.V_max = max(self.V)
        
        self.S_min = min(self.S)
        self.I_min = min(self.I)
        self.H_min = min(self.H)
        self.D_min = min(self.D)
        self.R_min = min(self.R)
        self.V_min = min(self.V)

        # Normalize
        self.S_hat = (self.S - self.S_min) / (self.S_max - self.S_min)
        self.I_hat = (self.I - self.I_min) / (self.I_max - self.I_min)
        self.H_hat = (self.H - self.H_min) / (self.H_max - self.H_min)
        self.D_hat = (self.D - self.D_min) / (self.D_max - self.D_min)
        self.R_hat = (self.R - self.R_min) / (self.R_max - self.R_min)
        self.V_hat = (self.V - self.V_min) / (self.V_max - self.V_min)

    

        # Neural network for SIHRDV model
        self.net_sidr = self.Net_sidr()
        self.params = list(self.net_sidr.parameters())

    

    # Neural network architecture
    class Net_sidr(nn.Module):  # input = [t]
        def __init__(self):
            super(DINN.Net_sidr, self).__init__()

            # Define hidden layers as a sequential block
            self.hidden_layers = nn.Sequential(
                nn.Linear(1, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20),
                nn.Linear(20, 20), nn.ReLU(), nn.BatchNorm1d(20)
            )

            # Define output layers
            self.out = nn.Linear(20, 6)  # Outputs for S, I, H, D, R, V
            self.param_out = nn.Linear(20, 7)  # 7 parameters: beta, delta, gammaR, gammaH, nuR, nuD, eta

            # Apply proper weight initialization
            self._initialize_weights()

        def _initialize_weights(self):
            """Initialize weights properly using He initialization for ReLU and Xavier for outputs."""
            for layer in self.modules():
                if isinstance(layer, nn.Linear):
                    init.kaiming_uniform_(layer.weight, nonlinearity='relu')  # He init for ReLU
                    init.zeros_(layer.bias)  # Set biases to 0

            # Xavier init for final output layers
            init.xavier_uniform_(self.out.weight)
            init.zeros_(self.out.bias)
            init.xavier_uniform_(self.param_out.weight)
            init.zeros_(self.param_out.bias)

        def forward(self, t_batch):
            """Forward pass through the network."""
            x = self.hidden_layers(t_batch)  # Pass through hidden layers
            sidr = self.out(x)  # Predict S, I, H, D, R, V
            params = self.param_out(x)  # Predict model parameters
            return sidr, params


        
    def net_f(self, t_batch, compute_gradients=True):
        t_batch.requires_grad_(compute_gradients) 
        sidr_hat, param_hat = self.net_sidr(t_batch)

        # Extract predictions from the network
        S_hat, I_hat, H_hat, D_hat, R_hat, V_hat = torch.unbind(sidr_hat, dim=1) 
        # Extract parameter predictions
        beta_hat, delta_hat, gammaR_hat, gammaH_hat, nuR_hat, nuD_hat, eta_hat = torch.unbind(param_hat, dim=1)
        
        # Apply constraints to parameters
        self.beta = torch.abs(torch.tanh(beta_hat))  
        self.delta = torch.abs(torch.tanh(delta_hat))
        self.gammaR = torch.abs(torch.tanh(gammaR_hat))
        self.gammaH = torch.abs(torch.tanh(gammaH_hat))
        self.nuR = torch.abs(torch.tanh(nuR_hat))
        self.nuD = torch.abs(torch.tanh(nuD_hat))
        self.eta = torch.abs(torch.tanh(eta_hat))
        
        if not compute_gradients:
            # If gradients are not needed, return only the predictions
            return None, None, None, None, None, S_hat, I_hat, H_hat, D_hat, R_hat, V_hat

        # Compute gradients for S, I, H, D, R, V
        S_hat_t = torch.autograd.grad(S_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]
        I_hat_t = torch.autograd.grad(I_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]
        H_hat_t = torch.autograd.grad(H_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]
        D_hat_t = torch.autograd.grad(D_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]
        R_hat_t = torch.autograd.grad(R_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]
        V_hat_t = torch.autograd.grad(V_hat.sum(), t_batch, retain_graph=True, create_graph=True)[0]

        # Unnormalize predictions
        S = self.S_min + (self.S_max - self.S_min) * S_hat
        I = self.I_min + (self.I_max - self.I_min) * I_hat
        H = self.H_min + (self.H_max - self.H_min) * H_hat
        D = self.D_min + (self.D_max - self.D_min) * D_hat
        R = self.R_min + (self.R_max - self.R_min) * R_hat
        V = self.V_min + (self.V_max - self.V_min) * V_hat

        # Compute physical constraints f1_hat, f2_hat, etc.
        f1_hat = S_hat_t - (-self.beta * S_hat * I_hat / self.N - self.delta * V_hat + self.eta * R_hat)
        f2_hat = I_hat_t - (self.beta * S_hat * I_hat / self.N - self.gammaH * I_hat - self.gammaR * I_hat)
        f3_hat = H_hat_t - (self.gammaH * I_hat - self.nuD * H_hat - self.nuR * H_hat)
        f4_hat = D_hat_t - (self.nuD * H_hat)
        f5_hat = R_hat_t - (self.gammaR * I_hat + self.nuR * H_hat + self.delta * V_hat - self.eta * R_hat)
        
        return f1_hat, f2_hat, f3_hat, f4_hat, f5_hat, S_hat, I_hat, H_hat, D_hat, R_hat, V_hat

    def load(self,path1):
      # Load checkpoint
      try:
        checkpoint = torch.load(path1 + str(self.save)+'.pt') 
        #print('\nloading pre-trained model...')############
        self.load_state_dict(checkpoint['model'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler'])
        epoch = checkpoint.get('epoch', 0)
        self.losses = checkpoint.get('losses', [])
         

      except RuntimeError :
          print('changed the architecture, ignore')
          pass
      except FileNotFoundError:
          pass
    
    def find_lambda(self, adaptive_lambda, phy_loss, mse_loss, model, beta=0.9):
        """
        Dynamically adjusts the lambda (weight) for the MSE loss to balance with PHY loss.
        Parameters:
            adaptive_lambda: Current lambda value
            phy_loss: Physics-based loss
            mse_loss: Data loss
            beta: Momentum term for smooth updates (default: 0.9)
        Returns:
            Updated adaptive_lambda
        """
        # Compute gradients of PHY loss
        phy_grad = torch.autograd.grad(phy_loss, model.parameters(), retain_graph=True, allow_unused=True)
        # Compute gradients of MSE loss
        mse_grad = torch.autograd.grad(mse_loss, model.parameters(), retain_graph=True, allow_unused=True)

        # Filter out None gradients
        phy_grad_norm = torch.stack([g.norm() for g in phy_grad if g is not None]).max()
        mse_grad_norm = torch.stack([g.norm() for g in mse_grad if g is not None]).mean()
        
        
        # Log-space scaling to reduce magnitude difference
        lambda_ratio  = torch.exp(torch.log(grad_phy_norm) - torch.log(grad_mse_norm))

        # Momentum smoothing (EMA)
        adaptive_lambda = beta * adaptive_lambda + (1 - beta) * lambda_ratio

        return adaptive_lambda

    def train(self, n_epochs, path1, iteration, country_name, l2_lambda=1e-5,
          early_stopping_patience=15000, early_stopping_min_delta=1e-7):
        
        length = len(self.S_hat)
        train_end = int(length * 0.9)  # Use 90% for training
        val_start = train_end  # Validation starts from the end of the training set

        # Early stopping variables
        best_loss = float('inf')
        epochs_without_improvement = 0
        patience = early_stopping_patience
        
        
        
        # Prepare to track loss history
        loss_history = {
            'epoch': [],
            'iteration': [],
            'MSE_loss': [],
            'PHY_loss': [],
            'L2_loss': [],
            'TOT_loss': [],
            'val_loss': [],
            'lambda_mse': []
        }


        self.load(path1)  # Load the model if checkpoint exists

        MSE_loss = np.zeros(n_epochs)
        PHY_loss = np.zeros(n_epochs)
        L2_loss = np.zeros(n_epochs)
        TOT_loss = np.zeros(n_epochs)
        val_loss_history = np.zeros(n_epochs)

        print('\nStarting training...\n')

        best_val_loss = float('inf')  # For tracking the best validation loss for saving the model

        for epoch in range(n_epochs):
            
            S_train_list = []
            I_train_list = []
            H_train_list = []
            D_train_list = []
            R_train_list = []
            V_train_list = []

            
            # Training step
            self.optimizer.zero_grad()  # Reset gradients for the training step
            f1, f2, f3, f4, f5, S_train_pred, I_train_pred, H_train_pred, D_train_pred, R_train_pred, V_train_pred = self.net_f(self.t_batch[:train_end])

            # Compute losses on the training set 
            mse_loss_train = (F.mse_loss(self.S_hat[:train_end].float(), S_train_pred.float()) +
                              F.mse_loss(self.I_hat[:train_end].float(), I_train_pred.float()) +
                              F.mse_loss(self.H_hat[:train_end].float(), H_train_pred.float()) +
                              F.mse_loss(self.D_hat[:train_end].float(), D_train_pred.float()) +
                              F.mse_loss(self.R_hat[:train_end].float(), R_train_pred.float()) +
                              F.mse_loss(self.V_hat[:train_end].float(), V_train_pred.float()))


            phy_loss_train = (torch.mean(torch.square(f1)) +
                              torch.mean(torch.square(f2)) +
                              torch.mean(torch.square(f3)) +
                              torch.mean(torch.square(f4)) +
                              torch.mean(torch.square(f5)))
            
            # Append the unnormalized predictions to the lists after each training step
            S_train_list.append(self.S_min + (self.S_max - self.S_min) * S_train_pred) 
            I_train_list.append(self.I_min + (self.I_max - self.I_min) * I_train_pred)
            H_train_list.append(self.H_min + (self.H_max - self.H_min) * H_train_pred)
            D_train_list.append(self.D_min + (self.D_max - self.D_min) * D_train_pred)
            R_train_list.append(self.R_min + (self.R_max - self.R_min) * R_train_pred)
            V_train_list.append(self.V_min + (self.V_max - self.V_min) * V_train_pred)

            
            self.lambda_mse = self.find_lambda(self.lambda_mse, phy_loss_train, mse_loss_train, self.net_sidr, beta=0.9)

            
            # Compute the total loss
            l2_loss = sum(torch.sum(param ** 2) for param in self.net_sidr.parameters()) * l2_lambda
 
            total_loss_train = (self.lambda_mse * mse_loss_train + phy_loss_train + l2_loss)
            

            # Backpropagation and optimization
            total_loss_train.backward()
            torch.nn.utils.clip_grad_norm_(self.net_sidr.parameters(), max_norm=5)  # Gradient clipping
            self.optimizer.step()
            self.scheduler.step()
            
   
            # Track training losses
            MSE_loss[epoch] = mse_loss_train.detach().cpu().numpy()
            PHY_loss[epoch] = phy_loss_train.detach().cpu().numpy()
            L2_loss[epoch] = l2_loss.detach().cpu().numpy()
            TOT_loss[epoch] = total_loss_train.detach().cpu().numpy()
            
            S_val_list = []
            I_val_list = []
            H_val_list = []
            D_val_list = []
            R_val_list = []
            V_val_list = []


            # Validation step
            with torch.no_grad():
                _, _, _, _, _, S_val_pred, I_val_pred, H_val_pred, D_val_pred, R_val_pred, V_val_pred = self.net_f(self.t_batch[val_start:], compute_gradients=False)
                
                
                S_val_list.append(self.S_min + (self.S_max - self.S_min) * S_val_pred) 
                I_val_list.append(self.I_min + (self.I_max - self.I_min) * I_val_pred)
                H_val_list.append(self.H_min + (self.H_max - self.H_min) * H_val_pred)
                D_val_list.append(self.D_min + (self.D_max - self.D_min) * D_val_pred)
                R_val_list.append(self.R_min + (self.R_max - self.R_min) * R_val_pred)
                V_val_list.append(self.V_min + (self.V_max - self.V_min) * V_val_pred)

            
                mse_loss_val = (F.mse_loss(self.S_hat[val_start:].float(), S_val_pred.float()) +
                                F.mse_loss(self.I_hat[val_start:].float(), I_val_pred.float()) +
                                F.mse_loss(self.H_hat[val_start:].float(), H_val_pred.float()) +
                                F.mse_loss(self.D_hat[val_start:].float(), D_val_pred.float()) +
                                F.mse_loss(self.R_hat[val_start:].float(), R_val_pred.float()) +
                                F.mse_loss(self.V_hat[val_start:].float(), V_val_pred.float()))

                val_loss_history[epoch] = mse_loss_val.detach().cpu().numpy()

            # Early stopping logic
            if mse_loss_val < best_val_loss - early_stopping_min_delta:
                best_val_loss = mse_loss_val
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            # Record losses
            loss_history['epoch'].append(epoch)
            loss_history['iteration'].append(iteration)  # Track the iteration (i)
            loss_history['MSE_loss'].append(mse_loss_train.detach().cpu().numpy())
            loss_history['PHY_loss'].append(phy_loss_train.detach().cpu().numpy())
            loss_history['L2_loss'].append(l2_loss.detach().cpu().numpy())
            loss_history['TOT_loss'].append(total_loss_train.detach().cpu().numpy())
            loss_history['val_loss'].append(mse_loss_val.detach().cpu().numpy())
            loss_history['lambda_mse'].append(self.lambda_mse)
            
            if epoch % 500 == 0:
                print(f"[Epoch {epoch}/{n_epochs}] [MSE Loss: {mse_loss_train.detach().cpu().numpy()}] "f"[Physics Loss: {phy_loss_train.detach().cpu().numpy()}] "f"[lambda_mse: {self.lambda_mse}] "f"[L2 Loss: {l2_lambda}] "f"[Validation Loss: {mse_loss_val.detach().cpu().numpy()}]")


            

            # Early stopping
            if epochs_without_improvement >= patience:
                print(f"Early stopping at epoch {epoch + 1}. Best Validation Loss: {best_val_loss:.6f}")
                break

            # Save the model periodically
            if epoch % 5000 == 0:
                torch.save({
                    'epoch': epoch,
                    'model': self.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'scheduler': self.scheduler.state_dict(),
                }, path1 + str(self.save) + '.pt')
                self.save = 2 if self.save % 2 != 0 else 3
        

        # Save losses for the current iteration
        csv_path = os.path.join(path14, f"losses_{country_name}_{iteration}.csv")
        loss_df = pd.DataFrame(loss_history)

        # Write the DataFrame to a new file for this iteration
        loss_df.to_csv(csv_path, index=False)


        return S_train_list, I_train_list, H_train_list, D_train_list, R_train_list, V_train_list, S_val_list, I_val_list, H_val_list, D_val_list, R_val_list, V_val_list



SyntaxError: invalid syntax (2607045494.py, line 383)

In [8]:
path = path13
pop_DEU = 83369840.0
pop_FRA = 67813000.0
pop_ITA = 59037472.0

window_size= 90
data_DEU = prepare_data(path10, pop_DEU, window_size=window_size)
data_FRA = prepare_data(path11, pop_FRA, window_size=window_size)
data_ITA = prepare_data(path16, pop_ITA, window_size=window_size)

def net(path, n, m, epoch=50000):
    results = {}
    params_dict = {}

    for country_name, country_data in [("France", data_FRA), ("Italy", data_ITA), ("Germany", data_DEU)]:
        country_results = {}
        country_params = {}
        dinn_country = [None] * len(country_data)
        print(f"\n=== Starting training for country: {country_name}")


        

        for i in range(n, m):
            covid_cases = csvmaker(country_data, country_name)
            steps = datatotimesteps(covid_cases)
            print(f"\n=== Iteration {i} ===")
            
            country_data[i].t, country_data[i].S, country_data[i].I, country_data[i].H, country_data[i].D, country_data[i].R, country_data[i].V = steps

            path2 = os.path.join(path, country_name + "_" + str(i))
            dinn_country[i] = DINN(country_data[i].t, country_data[i].S, country_data[i].I, country_data[i].H, country_data[i].D, country_data[i].R, country_data[i].V, path2, 1)
            learning_rate = 1e-5
            optimizer = optim.Adam(dinn_country[i].params, lr=learning_rate, weight_decay=1e-4)  # Added weight_decay for regularization
            dinn_country[i].optimizer = optimizer
            scheduler = torch.optim.lr_scheduler.CyclicLR(dinn_country[i].optimizer, base_lr=1e-5, max_lr=1e-3, step_size_up=1000, mode="exp_range", gamma=0.85, cycle_momentum=False)
            dinn_country[i].scheduler = scheduler
            
            try:
                S_train_list, I_train_list, H_train_list, D_train_list, R_train_list, V_train_list, S_val_list, I_val_list, H_val_list, D_val_list, R_val_list, V_val_list = dinn_country[i].train(epoch, path2, iteration=i, country_name=country_name)
            except EOFError:
                if dinn_country[i].save == 2:
                    dinn_country[i].save = 3
                    S_train_list, I_train_list, H_train_list, D_train_list, R_train_list, V_train_list, S_val_list, I_val_list, H_val_list, D_val_list, R_val_list, V_val_list = dinn_country[i].train(epoch, path2,iteration=i,country_name=country_name)
                elif dinn_country[i].save == 3:
                    dinn_country[i].save = 2
                    S_train_list, I_train_list, H_train_list, D_train_list, R_train_list, V_train_list, S_val_list, I_val_list, H_val_list, D_val_list, R_val_list, V_val_list = dinn_country[i].train(epoch, path2, iteration=i, country_name=country_name)

            
            key = f"{country_name}_{i}"
            
            # Store both training and validation results
            country_results[key] = {
                "train": (S_train_list, I_train_list, H_train_list, D_train_list, R_train_list, V_train_list),
                "val": (S_val_list, I_val_list, H_val_list, D_val_list, R_val_list, V_val_list)
            }
            
            
            # Store model parameters
            country_params[key] = {
                "beta": dinn_country[i].beta.tolist() if isinstance(dinn_country[i].beta, torch.Tensor) else dinn_country[i].beta,
                "delta": dinn_country[i].delta.tolist() if isinstance(dinn_country[i].delta, torch.Tensor) else dinn_country[i].delta,
                "gammaR": dinn_country[i].gammaR.tolist() if isinstance(dinn_country[i].gammaR, torch.Tensor) else dinn_country[i].gammaR,
                "gammaH": dinn_country[i].gammaH.tolist() if isinstance(dinn_country[i].gammaH, torch.Tensor) else dinn_country[i].gammaH,
                "nuR": dinn_country[i].nuR.tolist() if isinstance(dinn_country[i].nuR, torch.Tensor) else dinn_country[i].nuR,
                "nuD": dinn_country[i].nuD.tolist() if isinstance(dinn_country[i].nuD, torch.Tensor) else dinn_country[i].nuD,
                "eta": dinn_country[i].eta.tolist() if isinstance(dinn_country[i].eta, torch.Tensor) else dinn_country[i].eta,
            }
            

        results[country_name] = country_results
        params_dict[country_name] = country_params

    return results, params_dict



FileNotFoundError: [Errno 2] No such file or directory: './data/DEU_Omicron_check.csv'

In [ ]:

n=0
m=388#len(g)

e=30001


results = net (path,n,m,epoch=e)

In [ ]:
#results

In [ ]:
data_frames = {}

for country, iterations in results[1].items():
    if country not in data_frames:
        # Initialize a DataFrame with columns for iterations and rows for parameters
        data_frames[country] = pd.DataFrame(
            columns=list(range(n, m)), 
            index=["Beta", "Delta", "GammaR", "GammaH", "NuR", "NuD", "Eta"],
            dtype=object
        )

    for iteration, params in iterations.items():
        _, index = iteration.split('_')  # Extract the iteration index
        index = int(index)

        # Add the parameter lists to the corresponding column in the DataFrame
        for param_name, param_values in params.items():
            data_frames[country][int(index)] = params.values()
            #data_frames[country].at[param_name.capitalize(), index] = param_values

# Save each country's DataFrame to a CSV file
for country, df in data_frames.items():
    df.to_csv(f"{path14}/Parameters_{n}_{m}_{country}.csv", index=True)


In [ ]:
# Initialize dictionaries for predictions
S_pred_list_train = {}
I_pred_list_train = {}
H_pred_list_train = {}
D_pred_list_train = {}
R_pred_list_train = {}
V_pred_list_train = {}

S_pred_list_val = {}
I_pred_list_val = {}
H_pred_list_val = {}
D_pred_list_val = {}
R_pred_list_val = {}
V_pred_list_val = {}

S_lists_train = {}
I_lists_train = {}
H_lists_train = {}
D_lists_train = {}
R_lists_train = {}
V_lists_train = {}

S_lists_val = {}
I_lists_val = {}
H_lists_val = {}
D_lists_val = {}
R_lists_val = {}
V_lists_val = {}

countries = ["France", "Italy", "Germany"]

# Iterate through each country
for country in countries:
    # Initialize the lists for each country's training and validation predictions
    S_pred_list_train[country] = []
    I_pred_list_train[country] = []
    H_pred_list_train[country] = []
    D_pred_list_train[country] = []
    R_pred_list_train[country] = []
    V_pred_list_train[country] = []

    S_pred_list_val[country] = []
    I_pred_list_val[country] = []
    H_pred_list_val[country] = []
    D_pred_list_val[country] = []
    R_pred_list_val[country] = []
    V_pred_list_val[country] = []

    country_data = results[0][country]

    for i in range(n, m):
        key = f"{country}_{i}"

        if key in country_data:
            train_pred = country_data[key]['train']
            val_pred = country_data[key]['val']

            # Append training predictions
            S_pred_list_train[country].append(train_pred[0])
            I_pred_list_train[country].append(train_pred[1])
            H_pred_list_train[country].append(train_pred[2])
            D_pred_list_train[country].append(train_pred[3])
            R_pred_list_train[country].append(train_pred[4])
            V_pred_list_train[country].append(train_pred[5])

            # Append validation predictions
            S_pred_list_val[country].append(val_pred[0])
            I_pred_list_val[country].append(val_pred[1])
            H_pred_list_val[country].append(val_pred[2])
            D_pred_list_val[country].append(val_pred[3])
            R_pred_list_val[country].append(val_pred[4])
            V_pred_list_val[country].append(val_pred[5])

    # Convert the tensors to lists for both training and validation sets
    S_list_train = []
    for tensor_sublist in S_pred_list_train[country]:
        converted_sublist = [tensor.tolist() for tensor in tensor_sublist]
        S_list_train.append(converted_sublist)
    S_lists_train[country] = S_list_train

    S_list_val = []
    for tensor_sublist in S_pred_list_val[country]:
        converted_sublist = [tensor.tolist() for tensor in tensor_sublist]
        S_list_val.append(converted_sublist)
    S_lists_val[country] = S_list_val

    # Repeat the above conversion for all other categories (I, H, D, R, V)
    I_list_train = []
    for tensor_sublist in I_pred_list_train[country]:
        converted_sublist = [tensor.tolist() for tensor in tensor_sublist]
        I_list_train.append(converted_sublist)
    I_lists_train[country] = I_list_train

    I_list_val = []
    for tensor_sublist in I_pred_list_val[country]:
        converted_sublist = [tensor.tolist() for tensor in tensor_sublist]
        I_list_val.append(converted_sublist)
    I_lists_val[country] = I_list_val

    # For H, D, R, V for both train and validation
    # Training sets
    H_lists_train[country] = [[tensor.tolist() for tensor in sublist] for sublist in H_pred_list_train[country]]
    D_lists_train[country] = [[tensor.tolist() for tensor in sublist] for sublist in D_pred_list_train[country]]
    R_lists_train[country] = [[tensor.tolist() for tensor in sublist] for sublist in R_pred_list_train[country]]
    V_lists_train[country] = [[tensor.tolist() for tensor in sublist] for sublist in V_pred_list_train[country]]

    # Validation sets
    H_lists_val[country] = [[tensor.tolist() for tensor in sublist] for sublist in H_pred_list_val[country]]
    D_lists_val[country] = [[tensor.tolist() for tensor in sublist] for sublist in D_pred_list_val[country]]
    R_lists_val[country] = [[tensor.tolist() for tensor in sublist] for sublist in R_pred_list_val[country]]
    V_lists_val[country] = [[tensor.tolist() for tensor in sublist] for sublist in V_pred_list_val[country]]

# Save the predictions for both training and validation into CSV files
country_dfs_train = {}
country_dfs_val = {}

for country in countries:
    # Training DataFrame
    df_train = pd.DataFrame([S_lists_train[country], I_lists_train[country], H_lists_train[country], D_lists_train[country], R_lists_train[country], V_lists_train[country]],
                            index=["S_pred_list_train", "I_pred_list_train", "H_pred_list_train", "D_pred_list_train", "R_pred_list_train", "V_pred_list_train"],
                            columns=list(range(n, m)))
    country_dfs_train[country] = df_train

    # Validation DataFrame
    df_val = pd.DataFrame([S_lists_val[country], I_lists_val[country], H_lists_val[country], D_lists_val[country], R_lists_val[country], V_lists_val[country]],
                          index=["S_pred_list_val", "I_pred_list_val", "H_pred_list_val", "D_pred_list_val", "R_pred_list_val", "V_pred_list_val"],
                          columns=list(range(n, m)))
    country_dfs_val[country] = df_val

# Save to CSV for both training and validation results
for country, df_train in country_dfs_train.items():
    df_train.to_csv(path14 + f"Predictions_train_{n}_{m}_{country}.csv")

for country, df_val in country_dfs_val.items():
    df_val.to_csv(path14 + f"Predictions_val_{n}_{m}_{country}.csv")
